In [ ]:
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import folium
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import math
import random 
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

#colour pallette
cnf = '#393e46'
dth = '#ff2e63'
rec = '#21bf73'
act = '#fe9801'

# Dataset Preparation

In [ ]:
import plotly as py
py.offline.init_notebook_mode(connected = True)
import os

In [ ]:
df = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/covid_19_data_cleaned.csv', parse_dates=['Date'])
country_daywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/country_daywise.csv')
countrywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/countrywise.csv')
daywise = pd.read_csv('Covid-19-Preprocessed-Dataset/preprocessed/daywise.csv')

In [ ]:
df['Province/State'] = df['Province/State'].fillna("")
df.head()

In [ ]:
country_daywise

In [ ]:
countrywise

In [ ]:
daywise

In [ ]:
confirmed = df.groupby('Date').sum()['Confirmed'].reset_index()
confirmed.head()

In [ ]:
recovered = df.groupby('Date').sum()['Recovered'].reset_index()
recovered.head()

In [ ]:
deaths = df.groupby('Date').sum()['Deaths'].reset_index()
deaths.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.query('Country == "US"')

## Worldwide Total confirmed, Recovered, and Deaths

In [ ]:
confirmed.tail()

In [ ]:
recovered.tail()

In [ ]:
deaths.tail()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = confirmed['Date'], y = confirmed['Confirmed'], mode = 'lines+markers', name = 'Confirmed', line = dict(color = 'Orange', width = 4)))
fig.add_trace(go.Scatter(x = recovered['Date'], y = recovered['Recovered'], mode = 'lines+markers', name = 'Recovered', line = dict(color = 'Green', width = 4)))
fig.add_trace(go.Scatter(x = deaths['Date'], y = deaths['Deaths'], mode = 'lines+markers', name = 'Deaths', line = dict(color = 'Red', width = 4)))
fig.update_layout(title = 'Worldwide Covid-19 Cases', xaxis_tickfont_size = 14, yaxis = dict(title = 'Number of Cases'))
fig.show()

##  Cases Density Animation on World Map

In [ ]:
df.info()

In [ ]:
df['Date'] = df['Date'].astype(str)

In [ ]:
df.info()

In [ ]:
fig = px.density_mapbox(df, lat = 'Lat', lon = 'Long', hover_name = 'Country', hover_data = ['Confirmed', 'Recovered', 'Deaths'],animation_frame='Date', color_continuous_scale='Portland', radius = 7, zoom = 0, height=700)
fig.update_layout(title = 'Worldwide Covid 19 Cases with Time laps')
fig.update_layout(mapbox_style = 'open-street-map', mapbox_center_lon = 0)

fig.show()

## Cases Over the Time with Area Plot

In [ ]:
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp

In [ ]:
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)
temp

In [ ]:
temp = df.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop = True)

tm = temp.melt(id_vars = 'Date', value_vars = ['Active', 'Deaths', 'Recovered', 'Confirmed'])
fig = px.treemap(tm, path = ['variable'], values = 'value', height = 350, width = 1000, color_discrete_sequence=[act, rec, dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()

In [ ]:
# changing date column 
temp = df.groupby('Date').mean()
temp

In [ ]:
# bring the date column back to normal
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()

temp = temp.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths', 'Active'], var_name = 'Case', value_name = 'Count')


fig = px.area(temp, x = 'Date', y = 'Count', color= 'Case', height= 800, title = 'Cases over time', color_discrete_sequence=[rec, dth,  act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()


# Folium Maps

In [ ]:
temp = df[df['Date']==max(df['Date'])]

m = folium.Map(location=[0, 0], titles='cartodbpositron', min_zoom = 1, max_zoom=4, zoom_start=1)
               
for i in range(0, len(temp)):
    folium.Circle(location=[temp.iloc[i]['Lat'], temp.iloc[i]['Long']], color = 'crimson', fill = 'crimson', 
                 tooltip = '<li><bold> Country: ' + str(temp.iloc[i]['Country'])+
                             '<li><bold> Province: ' + str(temp.iloc[i]['Province/State'])+
                              '<li><bold> Confirmed: ' + str(temp.iloc[i]['Confirmed'])+
                               '<li><bold> Deaths: ' + str(temp.iloc[i]['Deaths']),
radius = int(temp.iloc[i]['Confirmed'])**0.5).add_to(m)
   
m

## Confirmed Cases with Choropleth Map

In [ ]:
fig = px.choropleth(country_daywise, locations= 'Country', locationmode='country names', color = country_daywise['Confirmed'],
                   hover_name ='Country', animation_frame = country_daywise['Date'],
                   title = 'Cases over time', color_continuous_scale=px.colors.sequential.Inferno)

fig.update(layout_coloraxis_showscale = True)
fig.show()

## Deaths and Recoveries per 100 Cases

In [ ]:
fig_c = px.bar(daywise, x = 'Date', y = 'Confirmed', color_discrete_sequence=[act])
fig_d = px.bar(daywise, x = 'Date', y = 'Deaths', color_discrete_sequence=[dth])

fig = make_subplots(rows = 1, cols = 2, shared_xaxes=False, horizontal_spacing=0.1,
                  subplot_titles=('Confirmed Cases', 'Deaths Cases'))

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.update_layout(height = 500)

fig.show()

## Confirmed and Death Cases with Static colormap

In [ ]:
daywise.head()

In [ ]:
fig_c = px.choropleth(countrywise, locations='Country', locationmode='country_names',
                     color = np.log(countrywise['Confirmed']), hover_name = 'Country',
                     hover_data = ['Confirmed'])

temp = countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp, locations ='Country', locationmode='country_names',
                     color = np.log(temp['Deaths']), hover_name = 'Country',
                     hover_data = ['Deaths'])

fig = make_subplots(rows = 1, cols = 2, subplot_titles= ['Confirmed', 'Deaths'],
                    specs = [[{'type': 'choropleth'}, {'type': 'choropleth'}]])

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)

fig.show()

In [ ]:
fig_c = px.choropleth(countrywise, locations='Country', locationmode='country_names',
                     color = np.log(countrywise['Confirmed']), hover_name = 'Country',
                     hover_data = (['Confirmed'])
temp = countrywise[countrywise['Deaths']>0]
fig_d = px.choropleth(temp, locations ='Country', locationmode='country_names',
                     color = np.log(temp['Deaths']), hover_name = 'Country',
                     hover_data = ['Deaths'])

fig = make_subplots(rows = 1, cols = 2, subplot_titles= ['Confirmed', 'Deaths'],
                   specs = [[{'type': 'choropleth'}, {'type': 'choropleth'}]])

fig.add_trace(fig_c['data'][0], row = 1, col = 1)
fig.add_trace(fig_d['data'][0], row = 1, col = 2)
fig.update(layout_coloraxis_showscale = False)
fig.show()

In [ ]:
full_latest = df[df['Date'] == max(df['Date'])]

fig = px.treemap(full_latest.sort_values(by = 'Confirmed', ascending = False).reset_index(drop = True),
                 path = ['Contry', 'Province/State'], values = 'Confirmed', height = 700,
                 title = 'Number of Confirmed Cases',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()